# 의사결정나무 모델 (Decision Tree Model)

- 데이터를 특정 기준에 따라 분할하고 트리구조를 구성해 데이터를 분류/회귀
- 장점 :
  - 굉장히 직관적이고, 트리 모델 학습의 구조를 이해하기가 쉽다
  - 비모수데이터(정규성을 따르지 않는 형태의 데이터, 이상치가 많은 데이터)에서도 잘 작동 / 전처리가 용이함
  - 데이터 개수에도 영향이 적다 / 대용량 데이터에 대해서 매우 잘 작동

- 단점 :
  - 과적합 Overfitting이 매우 쉽게 잘 발생 / 하이퍼파라미터 튜닝이 필수적
  - 분류 경계에서 오류 발생의 가능성이 높다
 
- Tree 구조 :
  - Node : 데이터를 특정 기준에 따라 분할하는 역할
  - Depth : 각 Node가 특정 조건에 대해 분할 되며 발생하는 층
  - Leaf (Terminal Node) : 분할이 모두 끝난 뒤, 맨 아래층 (더이상 분할이 불가능한) Node
  - Branch : Root Node부터 맨 아래 Leaf의 하나의 분류 기준들이 있는 줄기

- Tree 최적화 기법 : 데이터를 잘 분할 시킬 수 있게끔 계산하는 최적화 알고리즘
  - Gini (불순도) : 데이터가 얼마나 섞여있지 않고 순도 높게 유지되는지에 대한 지표
    - Gini 높다 : 불순물이 많다 -> 데이터가 섞여 있는 상태
    - Gini 낮다 : 순도가 높다 -> 데이터가 명확히 분류되어 있는 상태
  - Entropy (무질서도) : 데이터의 무질서 정도를 나타내는 지표 (로그함수)
    - Entropy 낮다 : 질서 정연한 상테 / 에너지가 많이 들이지 않아도 되는 상태 (이미 에너지가 들어가서 질서 정연했으니 에너지 더 들일 필요XX)
    - Entropy 높다 : 무질서한 상태 / 에너지가 많이 들어가야 하는 상태
 
  - Log Loss : 분류에서의 손실함수를 이용해 데이터가 얼마나 잘 분류됐는지를 계산하는 지표

In [37]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as express

In [54]:
df1 = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\14_Data.csv')
print(df1.shape)
df1.info()

(26283, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26283 entries, 0 to 26282
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   방송Code    26283 non-null  float64
 1   채널        26283 non-null  object 
 2   소요분       26283 non-null  float64
 3   가중분       26283 non-null  float64
 4   방송구분      26283 non-null  object 
 5   프로그램명     26283 non-null  object 
 6   상품ID      26283 non-null  int64  
 7   상품명       26283 non-null  object 
 8   매입과세구분    26283 non-null  object 
 9   상품목표취급금액  26283 non-null  int64  
 10  상품목표주문금액  26283 non-null  int64  
 11  판매단가      26283 non-null  int64  
 12  수수료율      26283 non-null  float64
 13  상품주문수량    26283 non-null  int64  
 14  상품주문금액    26283 non-null  int64  
 15  상품취소수량    26283 non-null  int64  
 16  상품취소금액    26283 non-null  int64  
 17  ARS금액     26283 non-null  int64  
 18  매입형태      26283 non-null  object 
 19  배송방식      26283 non-null  object 
 20  상품소요분     26283 

In [55]:
!pip install --upgrade imblearn

In [60]:
from sklearn.model_selection import train_test_split #
from imblearn.pipeline import make_pipeline # 파이프라인 구성
from sklearn.compose import make_column_transformer # 분할처리 때 필요(병렬로 배치)

from sklearn.impute import SimpleImputer # 결측값 처리
from sklearn.preprocessing import MinMaxScaler # 스케일링
from sklearn.preprocessing import OneHotEncoder # 인코딩

from imblearn.combine import SMOTEENN # 불균형데이터처리
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV # 교차검증&매개변수 튜닝
from sklearn.metrics import classification_report # 평가

In [61]:
df1['방송연도'] = pd.to_datetime(df1['방송시작시간']).dt.year
df1['방송월'] = pd.to_datetime(df1['방송시작시간']).dt.month
df1['방송일'] = pd.to_datetime(df1['방송시작시간']).dt.day
df1['방송요일'] = pd.to_datetime(df1['방송시작시간']).dt.day_name()

In [64]:
df1['목표달성액'] =   df1['상품목표주문금액'] - df1['상품주문금액']
cond1 = df1['목표달성액'] > 0
df1.loc[cond1, '목표달성여부'] = 1
df1.loc[~cond1, '목표달성여부'] = 0
df1['목표달성여부'].value_counts()

목표달성여부
1.0    16333
0.0     9950
Name: count, dtype: int64

In [65]:
X = df1[['소요분','방송구분','판매단가','ARS금액','수수료율','방송요일','방송월']]
Y = df1['목표달성여부']

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random_state = 1234)

In [67]:
# 문자는 문자끼리, 숫자는 숫자끼리 처리될수있도록 식별할 수 있게, 문자숫자 이름 리스트를 구성
numeric_list = X.describe().columns
category_list = X.describe(include='object').columns

In [68]:
# 전처리 파이프를 구성
numeric_pipe = make_pipeline(SimpleImputer(strategy='median'), MinMaxScaler())
category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())
prepro_pipe = make_column_transformer((numeric_pipe,numeric_list),(category_pipe,category_list))
model_pipe = make_pipeline(prepro_pipe, SMOTEENN(), DecisionTreeClassifier())
model_pipe.fit(X_train,Y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  Index(['소요분', '판매단가', 'ARS금액', '수수료율', '방송월'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['방송구분', '방송요일'], dtype='object'))])),
                ('smoteenn', SMOTEENN()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

- Tree Model Hyper-Parameter
  - criterion : 분류 최적화 알고리즘 선택 ('gini','entropy','log_loss')
  - **max_depth** : 트리의 최대 깊이를 제한 (기본값 None / 모두 분류가 될 때 까지 층을 형성)
  - **min_samples_split** : 노드를 분할 하기 위한 최소 데이터 수 (기본값 2) 
  - **min_samples_leaf** : Leaf 노드가 갖는 최소 데이터 수
  - class_weight : 클래스의 가중치를 부여할지 말지에 대한 매개변수 (None / 'balanced
  - max_lead_nodes : 최대 노드 수를 제한

In [78]:
hyperparameter = {'decisiontreeclassifier__criterion' : ['gini','entropy'], # 2
                  'decisiontreeclassifier__max_depth' : range(5,20), # 15
                  'decisiontreeclassifier__class_weight' : [None,'balanced'], # 2
                  'decisiontreeclassifier__min_samples_leaf' : [20,50,100]} # 3

grid_model = GridSearchCV(model_pipe, param_grid=hyperparameter, cv=3, scoring='f1', n_jobs=-1)
grid_model.fit(X_train,Y_train)
best_model = grid_model.best_estimator_

- Feature Importance (변수 중요도)
  - 분류를 수행함에 있어, 각 X가 얼마나 분류에 기여하였는가
  - 회귀분석의 회귀계수와는 개념이 다르다!
  - 변수중요도가 높다고 해서 Y값이 직접적으로 변하지는 않음 (습도가 8일때 분류가 잘되었다고 해서 습도가 Y값에 가장중요한건 아님. 습도가 8일때가 중요한거지)
  - 어떤 인자가 중요한지 찾기 위해 트리구조를 만들었다? 이건 안됨 -> 이 인자가 중요하네요~유의할필요가있습니다 라고 까지는 표현 가능

In [77]:
df_importance = pd.DataFrame()
df_importance['X'] = best_model[0].get_feature_name_out()

AttributeError: 'ColumnTransformer' object has no attribute 'get_feature_name_out'

In [ ]:
df_importance['importance'] = best_model['decisiontreeclassifier'].feature_importances_

In [ ]:
df_importance